In [5]:
import re
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from string import punctuation
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
from tensorflow.keras.layers import Input,Dense,LSTM
from tensorflow.keras.regularizers import l2
from keras.preprocessing import sequence
from keras.layers import concatenate,Flatten,Embedding,Dropout
from keras import Model
from tensorflow.python.keras.callbacks import TensorBoard
import pickle
import joblib
from tensorflow import keras
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [96]:
def fn1(df):
  #df = df.drop(['id', 'location'] , axis = 1)

  def clean_text(text):
  
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"  ","",text)
    """Removing website and HTML tags and punctiation"""
    text = re.sub(r"www" , "", text)
    text = re.sub(r"com" , "", text)
    text = re.sub(r"http" , "", text)
    text = re.sub(r"https" , "", text)
    text = re.sub(r"&amp" , "", text)
    #text = re.sub(r"??" , "", text)
    text = re.sub(r"-()\"#/@;:<>{}`+=~|.!?,", "", text)
  
    return text

  def stopword_removal(text):
      stop = [i for i in stopwords.words("english") if i != "not"]
      words = []
      for i in text.split():
        if i not in stop:
          words.append(i)
      return " ".join(words)

  def stem(text):
      return ' '.join([PorterStemmer().stem(i) for i in text.split()])

  for i in range(len(df.text)):
    df.iloc[i , 1] = clean_text(df.iloc[i , 1])
    df.iloc[i , 1] = stopword_removal(df.iloc[i , 1])
    df.iloc[i , 1] = stem(df.iloc[i , 1])
  
  df['keyword'].replace('', np.nan, inplace=True)
  df.dropna(subset=['keyword'], inplace=True)
  
  with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
  
  tokenized  = tokenizer.texts_to_sequences(df.text)
  train_padded = pad_sequences(tokenized, maxlen = 50  , padding='post' )

  enc = joblib.load('enc.pkl')
  keyword = enc.transform(df['keyword'].values.reshape(-1,1))

  model = keras.models.load_model("my_h5_model.h5")

  pred = model.predict([train_padded ,    keyword] )
  return pred , df.target

In [104]:
df = pd.read_csv("/content/Twitter_processed.csv")
x = df.iloc[ 6500: , 1:]

pred , y = fn1(x)

33/33 [==============================] - 1s 35ms/step


X is unseen data

In [105]:
for i in range(len(pred)):
  if pred[i][0] >= 0.5:
    pred[i][0] = 1
  else:
    pred[i][0] = 0

In [108]:
#y = df.target
print("Acuracy", round(accuracy_score(y, pred),2))

Acuracy 0.83


In [ ]:
def fn2(df):
  pred , y = fn1(df)
  for i in range(len(pred)):
    if pred[i][0] >= 0.5:
      pred[i][0] = 1
    else:
      pred[i][0] = 0
  #y = df.target
  print("Acuracy", round(accuracy_score(y, pred),2))


In [116]:
fn2(x)

33/33 [==============================] - 1s 35ms/step
Acuracy 0.83
